In [1]:
from torchvision import datasets
data_path = '../temp'
c10 = datasets.CIFAR10(data_path, train=True, download=True)
c10_val = datasets.CIFAR10(data_path, train=False, download=True)

Files already downloaded and verified
Files already downloaded and verified


In [2]:
from torchvision import transforms
to_tensor = transforms.ToTensor()
label_map = {0: 0, 2: 1}
c2 = [(to_tensor(img), label_map[label])
                 for img, label in c10
                 if label in [0, 2]]
c2_val = [(to_tensor(img), label_map[label])
                 for img, label in c10_val
                 if label in [0, 2]]

In [9]:
import torch
import torch.nn.functional as F
import torch.nn as nn

In [4]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(8*8*8, 32)
        self.fc2 = nn.Linear(32, 2)
    
    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = out.view(-1, 8*8*8)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out

In [7]:
img = c2[0][0]

In [5]:
model = Net()

In [16]:
img.shape

torch.Size([3, 32, 32])

In [10]:
model(img)

tensor([[0.0839, 0.1400]], grad_fn=<AddmmBackward0>)

In [11]:
loss_fn = nn.CrossEntropyLoss()

In [12]:
imgs = torch.stack([img[0].unsqueeze(0) for img in c2]).squeeze(1)

In [13]:
imgs.shape

torch.Size([10000, 3, 32, 32])

In [32]:
outputs = model(imgs)

In [33]:
outputs.shape

torch.Size([10000, 2])

In [14]:
labels = torch.tensor([img[1] for img in c2])

In [36]:
loss = loss_fn(outputs, labels)

In [37]:
loss

tensor(0.7061, grad_fn=<NllLossBackward0>)

In [39]:
2.3 ** (-0.7)

0.5582008773935958

In [16]:
trainer = torch.optim.SGD(model.parameters(), lr=0.01)

In [48]:
for i in range(500):
    outputs = model(imgs)
    loss = loss_fn(outputs, labels)
    
    trainer.zero_grad()
    loss.backward()
    trainer.step()
    
    if i%5 == 0:
        print(i, loss, evaluate_train(outputs), evaluate_val())

0 tensor(0.4664, grad_fn=<NllLossBackward0>) 0.7965 0.8015
5 tensor(0.4659, grad_fn=<NllLossBackward0>) 0.7967 0.803
10 tensor(0.4655, grad_fn=<NllLossBackward0>) 0.7968 0.8025
15 tensor(0.4651, grad_fn=<NllLossBackward0>) 0.7973 0.8025
20 tensor(0.4647, grad_fn=<NllLossBackward0>) 0.7975 0.8025
25 tensor(0.4643, grad_fn=<NllLossBackward0>) 0.7977 0.8025
30 tensor(0.4638, grad_fn=<NllLossBackward0>) 0.7981 0.802
35 tensor(0.4634, grad_fn=<NllLossBackward0>) 0.7984 0.802
40 tensor(0.4630, grad_fn=<NllLossBackward0>) 0.798 0.802
45 tensor(0.4626, grad_fn=<NllLossBackward0>) 0.798 0.8025
50 tensor(0.4621, grad_fn=<NllLossBackward0>) 0.7981 0.802
55 tensor(0.4617, grad_fn=<NllLossBackward0>) 0.7982 0.802
60 tensor(0.4613, grad_fn=<NllLossBackward0>) 0.7982 0.803
65 tensor(0.4609, grad_fn=<NllLossBackward0>) 0.7981 0.803
70 tensor(0.4604, grad_fn=<NllLossBackward0>) 0.798 0.804
75 tensor(0.4600, grad_fn=<NllLossBackward0>) 0.7981 0.8045
80 tensor(0.4596, grad_fn=<NllLossBackward0>) 0.7987 0

In [23]:
labels_val = torch.tensor([img[1] for img in c2_val])
imgs_val = torch.stack([img[0].unsqueeze(0) for img in c2_val]).squeeze(1)

In [28]:
def evaluate_val():
    pred = model(imgs_val)
    _, p = torch.max(pred, dim=1)
    return int((labels_val == p).sum())/2000

In [29]:
evaluate_val()

0.6965

In [39]:
def evaluate_train(pred):
    _, p = torch.max(pred, dim=1)
    return int((labels == p).sum())/10000

In [40]:
outputs = model(imgs)

In [43]:
evaluate_train(outputs)

0.7423